In [ ]:
import torch
from PIL import Image
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

from data import normalize_image,CustomDataset,get_coord_rgb
from model import hMLP,DinerSiren,DINERMlp,newhMLP,di2Mlp,nomodMLP,nomodSirenMLP,newhMLPSiren
from utils import disp_img_train,residual_display,PSNR,save_image,get_svd_and_num_eigenvectors_above_threshold

import torch.nn as nn
import torch.optim as optim

In [ ]:
Device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
path='RGB_OR_1200x1200_00.png'
img=normalize_image(path).to(Device)
h,w=img.shape[1],img.shape[2]
#img=img[:3,:576,:576]
disp_img=img*0.5+0.5
plt.imshow(disp_img.permute(1,2,0).to('cpu').numpy())
h,w=disp_img.shape[1],disp_img.shape[2]
plt.show()

In [ ]:
import os
def result_func():
  img1=model(1)
  img1=img1.view(h,w,3).detach()
  img2=img.permute(1,2,0)
  img1=img1*0.5+0.5
  psnr =PSNR(img1, img2)

  # Save PSNR, model.d, and model_name in results.txt
  with open('results.txt', 'a') as file:
      file.write(f"{psnr:.2f},{model.d} \n")

  # Save the image with a similar name
  image_name = os.path.splitext(path)[0]
  image_extension = os.path.splitext(path)[1]
  new_image_name = f"{image_name}_psnr{psnr:.2f}_d={model.d}_{image_extension}"
  img2=img2.detach().to('cpu').numpy()
  save_image(img2,new_image_name,h,w,model,usemodel=False)
  return

In [ ]:
dataset = CustomDataset(path)
coord_x,rgb_values_x=get_coord_rgb(dataset)

In [ ]:
coord_x,rgb_values_x=coord_x.to(Device),rgb_values_x.to(Device)

In [ ]:
#!echo '' > results.txt

In [ ]:
!nvidia-smi

In [ ]:
l=[1,5,10,20,50,100,125,150]
for i in l:
  input_size  = 2  
  hidden_size = 64
  inhash_size = 300
  output_size = 3  
  model=newhMLP(input_size,hidden_size,output_size,1200,1200,d=i)
  model=model.to(Device)
  print('modulation_RELU',model.d,sum(p.numel() for p in model.parameters() if p.requires_grad) )

In [ ]:
model=DINERMlp(2,64,3,1200,1200)
print('modulation_RELU',sum(p.numel() for p in model.parameters() if p.requires_grad) )

In [ ]:
x=torch.randint(0,100,(1200,1200))
print(x[:5,:5])
x=(x-50)/50
print(x[:5,:5])
coord_x,rgb_values_x=torch.zeros(1200*1200,1),torch.zeros(1200*1200,1)
for i in range(1200):
    for j in range(1200):
        coord_x[i*1200+j,:]=i*1200+j
        #print(rgb_values_x[i*1200+j,:])
        rgb_values_x[i*1200+j,:]=x[i,j]

coord_x,rgb_values_x=coord_x.to(Device),rgb_values_x.to(Device)

In [ ]:
print(rgb_values_x)

In [ ]:
l=[70]
#l=[300,350,400,450]
#l=[250,300,350]
for i in l:
  input_size = 2  
  hidden_size = 64
  inhash_size = 300
  output_size = 1  
  model=newhMLP(input_size,hidden_size,output_size,1200,1200,d=i)
  model=model.to(Device)
  
  criterion1 = nn.L1Loss()
  criterion=nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)
  num_epochs = 10000

  for epoch in range(num_epochs):
      output = model(1)
      #output=torch.round(output)
      loss = criterion(output, rgb_values_x)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      """ if epoch_loss<lowest_loss:
          lowest_loss=epoch_loss
          torch.save(model.state_dict(), 'model.pth') """
          #save_image(img, f"image_epoch_{epoch}.png",h,w,model,usemodel=True)
          
  #result_func()
  print(loss.item())


In [ ]:
def f(x,model):
    model.eval()
    with torch.no_grad(): 
        z=model(1)
        z=z.view(1200,1200)
        y,z=x*50+50,z*50+50
        y=y.to(Device)
        z=torch.round(z)
        l=torch.pow(y-z,2)
        print(y,z)
        print(l)
        print(torch.max(l),torch.argmax(l))
        print(torch.mean(l))
    return l
l=f(x,model)    

In [ ]:
def count(l):
    count=0
    for i in l:
        for j in i:
            if j!=0:
                count+=1
    return count
((count(l)/(1200*1200)),count(l))

In [ ]:
z=z*50+50
loss=torch.mean(torch.round(z)-x.to(Device))
loss

In [ ]:
x=torch.randint(0,100,(1200,1200,3))
print(x.shape)


In [ ]:
print(modell)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1=nn.Linear(2,64)
        self.fc2=nn.Linear(64,64)
        self.fc3=nn.Linear(64,64)
        self.fc4=nn.Linear(64,3)
    def forward(self,x):
        return x
model=MLP()
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
img1=model(1)
img1=img1.view(h,w,3).detach()
img2=img.permute(1,2,0)
img1=img1*0.5+0.5
psnr =PSNR(img1, img2)

disp_img_train(model,1200,1200)

In [ ]:
result_func()

In [ ]:
# Create the MLP model
input_size = 2  # x and y coordinates
hidden_size = 64
inhash_size = 300
output_size = 3  # RGB values
model=nomodMLP(input_size, hidden_size, output_size,1200,1200)
model=model.to(Device)
# Define the loss function and optimizer
criterion1 = nn.L1Loss()
criterion=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
lowest_loss=2
num_epochs = 10000
for epoch in range(num_epochs):
    epoch_loss=0
    count=0

    output = model(1)
    loss = criterion(output, rgb_values_x)

    epoch_loss+=loss.item()
    count+=1

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
      #print(loss.item())
    epoch_loss=epoch_loss/count
    """if epoch_loss<lowest_loss:
        lowest_loss=epoch_loss
        torch.save(model.state_dict(), 'model.pth')"""

    if (epoch) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f},Lowest_loss: {lowest_loss:.4f}')
        disp_img_train(model,h,w)
        #save_image(img, f"image_epoch_{epoch}.png",h,w,model,usemodel=True)



In [ ]:
img1=model(1)
img1=img1.view(h,w,3).detach()
img2=img.permute(1,2,0)
img1=img1*0.5+0.5
psnr =PSNR(img1, img2)
psnr

In [ ]:
!nvidia-smi

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
disp_img_train(model)

In [ ]:
new_img=img*0.5+0.5
plt.imshow(new_img.permute(1,2,0).to('cpu').numpy())
plt.show()

In [ ]:
genr_image=model(1)
genr_image=genr_image.view(h,w,3)
genr_image=genr_image*0.5+0.5
print(f'PSNR: {PSNR(genr_image,img.permute(1,2,0))}')

In [ ]:

save(genr_image,"imagesiren_15_33.8.png")

In [ ]:
hash_im=imagefrom_hash(model.hash[:,0],h,w)
print(imagefrom_hash(model.hash[:,0],h,w).shape)

In [ ]:
hash1=torch.matmul(model.hash01,model.hash02)
hash2=torch.matmul(model.hash11,model.hash12)
hash3=torch.matmul(model.hash21,model.hash22)
hash4=torch.matmul(model.hash31,model.hash32)
hash11=torch.matmul(model.hash011,model.hash021)
hash21=torch.matmul(model.hash111,model.hash121)
hash31=torch.matmul(model.hash211,model.hash221)
hash41=torch.matmul(model.hash311,model.hash321)

In [ ]:
def get_svd_and_num_eigenvectors_above_threshold(matrix, threshold=0.5,Device='cpu'):
  U, S, V = torch.svd(matrix)
  matrix=torch.mm(torch.mm(U, torch.diag(S)), V.t())

  m = nn.Threshold(threshold,0)
  S=m(S)
  num_eigenvectors = torch.sum(S > threshold)
  new_matrix=torch.mm(torch.mm(U, torch.diag(S)), V.t()).to(Device)
  print(torch.dist(new_matrix,matrix.to(Device)))

  return new_matrix,num_eigenvectors,S

In [ ]:
#x=model.hash[:,0].unsqueeze(1)
m,n,S=get_svd_and_num_eigenvectors_above_threshold(hash41,0.05)
n,S[:100]

In [ ]:
with torch.no_grad():
  x=model.hash.data
  print(x.shape)
  m=m.view(x.shape,1)
  print(m.shape)
  model.hash.data=m
  disp_img_train(model)

In [ ]:
genr_image=model(1)
genr_image=genr_image.view(h,w,3)
genr_image=genr_image*0.5+0.5
print(f'PSNR,n: {PSNR(genr_image,img.permute(1,2,0))},{n}')

In [ ]:
x=torch.randn(1200,1200)
a,b=get_svd_and_num_eigenvectors_above_threshold(x,0.9)
b.sum(),a

In [ ]:
model.eval()
with torch.no_grad():
  genr_image=torch.zeros(h, w, 3).to(Device)
  hash=torch.matmul(model.hash01,model.hash02)
  h,w=hash.shape
  hash=hash.view(w*h,1)
  for x in range(h):
    for y in range(w):
      #genr_image[x,y,:]=model(torch.tensor([x,y],dtype=torch.float32),hash=False)
      #genr_image[x,y,:]=model(torch.cat((model.hash[w*x+y].unsqueeze(1),model.hash[w*x+y].unsqueeze(1)),dim=1),hash=False)
      #genr_image[x,y,:]=model(torch.tensor([x,y],dtype=torch.float32).to(Device))
      genr_image[x,y,:]=model(hash[w*x+y],hash=False)

  genr_image=genr_image*0.5+0.5
  plt.imshow(genr_image.detach().to('cpu').numpy())
  plt.show()